<a href="https://colab.research.google.com/github/FatemeZamanian/DeepLearning/blob/main/MaskRecognition/MaskRecognitionTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle

In [ ]:
!kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset

In [ ]:
!unzip -qq face-mask-12k-images-dataset.zip

In [ ]:
from tensorflow.keras import models,layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [ ]:
width=height=224
batch_size=32

In [ ]:
data_generator=ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
)
train_data=data_generator.flow_from_directory(
    "Face Mask Dataset/Train",
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
)

val_data=data_generator.flow_from_directory(
    "Face Mask Dataset/Validation",
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
)

test_data=data_generator.flow_from_directory(
    "Face Mask Dataset/Test",
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False,
)

Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 992 images belonging to 2 classes.


In [ ]:
model_r=tf.keras.applications.ResNet50V2(
    input_shape=(width,height,3),
    include_top=False,
    weights='imagenet',
    pooling='max'
)
# model_r.summary()

In [ ]:
for layer in model_r.layers[:-2]:
  layer.trainable=False

In [ ]:
model=tf.keras.Sequential([
        model_r,
        layers.Dense(2,activation='softmax'),
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001 ),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'],)

In [ ]:
model.fit(train_data,
          steps_per_epoch=train_data.samples/batch_size,
          validation_data=val_data,
          validation_steps=val_data.samples/batch_size,
          epochs=5,

)

Epoch 1/5
312/312 [==============================] - 104s 223ms/step - loss: 0.1031 - accuracy: 0.9838 - val_loss: 0.0257 - val_accuracy: 0.9962
Epoch 2/5
312/312 [==============================] - 67s 215ms/step - loss: 0.0288 - accuracy: 0.9962 - val_loss: 0.0166 - val_accuracy: 0.9937
Epoch 3/5
312/312 [==============================] - 67s 215ms/step - loss: 0.0167 - accuracy: 0.9973 - val_loss: 0.0096 - val_accuracy: 0.9975
Epoch 4/5
312/312 [==============================] - 67s 215ms/step - loss: 0.0181 - accuracy: 0.9967 - val_loss: 0.0056 - val_accuracy: 0.9987
Epoch 5/5
312/312 [==============================] - 67s 215ms/step - loss: 0.0035 - accuracy: 0.9991 - val_loss: 9.7532e-06 - val_accuracy: 1.0000


In [ ]:
model.evaluate(test_data)

31/31 [==============================] - 6s 199ms/step - loss: 2.8885e-05 - accuracy: 1.0000


[2.8885277060908265e-05, 1.0]

In [ ]:
model.save('maskrec.h5')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
Y_pred = model.predict(test_data)
y_pred = np.argmax(Y_pred, axis = 1)
print('confusion Matrix')
print(confusion_matrix(test_data.classes, y_pred))

target_names = list(test_data.class_indices.keys())
print('Classification Report')
print(classification_report(test_data.classes, y_pred, target_names=target_names))

confusion Matrix
[[483   0]
 [  0 509]]
Classification Report
              precision    recall  f1-score   support

    WithMask       1.00      1.00      1.00       483
 WithoutMask       1.00      1.00      1.00       509

    accuracy                           1.00       992
   macro avg       1.00      1.00      1.00       992
weighted avg       1.00      1.00      1.00       992

